In [1]:
import requests, json
import networkx as nx
from networkx.algorithms.smallworld import sigma, omega
import numpy as np
from collections import deque
from pyvis.network import Network
import matplotlib.pyplot as plt

# ***FASE 1***
Si costruisce il grafo delle dipendenze a partire dal seed _angular-cli_ e si salva in un file JSON: `grafo_diretto.json`.

In [2]:
def trova_dipendenze(package_name):
    
    URL = f"https://registry.npmjs.org/{package_name}/latest"
    try:
        response = requests.get(URL)
        if response.status_code != 200:
            return {}
        data = response.json()
        return data.get("dependencies", {})
    except Exception as e:
        print(f"Errore con {package_name}: {e}")
        return {}

In [3]:
def bfs_with_nx(seed_package):
    G = nx.DiGraph()
    coda = deque([(seed_package, 0)])  # Inizializza la coda con il seed
    visitati = set()

    while coda:
        nodo_corrente, livello = coda.popleft()

        if nodo_corrente in visitati:
            continue

        visitati.add(nodo_corrente)
        G.add_node(nodo_corrente, livello=livello)  # Aggiunge il nodo con attributi

        # Trova le dipendenze del nodo corrente
        dependencies = trova_dipendenze(nodo_corrente)
        for dep in dependencies.keys():
            if dep not in visitati:
                G.add_edge(nodo_corrente, dep)  # Aggiunge l'arco
                coda.append((dep, livello + 1))  # Aggiunge alla coda

    return G

In [4]:
nome_seed = "angular-cli"
#creazione del grafo
G = bfs_with_nx(nome_seed)

print(f"Totale pacchetti trovati: {G.number_of_nodes()}")
print(f"Totale archi trovati: {G.number_of_edges()}")

#salvo il grafo in un file
with open("grafo_diretto.json", "w") as f:
    json.dump(nx.node_link_data(G), f, indent=4)

Totale pacchetti trovati: 586
Totale archi trovati: 740


# ___FASE 2___
Si costruisce il _grafo indiretto_ ottenuto a paritre dal grafo diretto delle dipendenze a cui si aggiungono dei nodi artificiali. <br> Il grafo viene salvato in un file JSON: `grafo_artificiale.json`.

In [5]:
G2 = G.to_undirected() #creazione del grafo indiretto

In [6]:
def aggiungi_nodi_preferential(G, nodi_artificiali, m):

    for i in range(nodi_artificiali):
        nuovo_nodo = f"new_{i}" # Nome unico per il nuovo nodo
        G.add_node(nuovo_nodo)

        # Calcola probabilità basata sui gradi attuali
        somma_gradi = 2 * G.number_of_edges()
        packages = G.nodes()

        probabilita = [G.degree(package) / somma_gradi for package in packages]
        nodi_scelti = np.random.choice(packages, size = m, replace=False, p = probabilita)


        for nodo in nodi_scelti:
            G.add_edge(nuovo_nodo, nodo)


    return G

In [7]:
# Aggiungi nodi artificiali
nodi_artificiali = 400
m = 3 # Numero di connessioni per ogni nuovo nodo
G2 = aggiungi_nodi_preferential(G2, nodi_artificiali, m)

print(f"Grafo finale: {G2.number_of_nodes()} nodi, {G2.number_of_edges()} archi.")


Grafo finale: 986 nodi, 1940 archi.


In [8]:
#salvo il grafo in un file
with open("grafo_artificiale.json", "w") as f:
    json.dump(nx.node_link_data(G2), f, indent=4)

# ___FASE 3___
In questa fase vengono letti i file JSON contenenti i dati dei grafi e vengono convertiti in oggetti di NetworkX, `G` e `G2`. Per ognuno viene stampato il numero di nodi e archi.

In [9]:
with open("grafo_diretto.json", "r") as f:
    data = json.load(f)

G = nx.node_link_graph(data)  
print(f"Grafo diretto finale: {G.number_of_nodes()} nodi, {G.number_of_edges()} archi.")

with open("grafo_artificiale.json", "r") as f:
    data = json.load(f)

G2 = nx.node_link_graph(data)  
print(f"Grafo indiretto finale: {G2.number_of_nodes()} nodi, {G2.number_of_edges()} archi.")

Grafo diretto finale: 586 nodi, 740 archi.
Grafo indiretto finale: 986 nodi, 1940 archi.


# ___FASE 4___
In questa fase vengono visualizzati i due grafi in maniera _statica_ utilizzando il layout di Fruchterman-Reingold.

In [ ]:
plt.figure(figsize=(15, 15), dpi=400)
pos = nx.spring_layout(G, iterations=300, k = 1.05)
nx.draw(G, pos, with_labels=True, font_size=5.5, node_size=30, node_color="red", edge_color="gray", width=0.1)
plt.savefig("grafico1.png", dpi=500)
plt.show()

In [ ]:
plt.figure(figsize=(15, 15), dpi=400)
pos = nx.spring_layout(G2, iterations=300, k = 2)
nx.draw(G2, pos, with_labels=True, font_size=5.5, node_size=30, node_color="red", edge_color="gray", width=0.1)
plt.savefig("grafico2.png", dpi=500)
plt.show()

# ___FASE 5___
Utlizzando la libreria `pyvis` si visualizzano i due grafi in maniera _interattiva_.
- `grafo1_interattivo.html`
- `grafo2_interattivo.html`

In [ ]:
def calcola_quartili(gradi):

    q1 = np.percentile(gradi, 25) #primo quartile
    q2 = np.percentile(gradi, 50) #secondo quartile
    q3 = np.percentile(gradi, 75) #terzo quartile
    return q1, q2, q3

#funzione che assegna il colore al nodo sulla base del quartile di appartenenza
def colore_nodo(grado, q1, q2, q3):

    if grado <= q1:
        return "gray"
    elif grado <= q2:
        return "blue"
    elif grado <= q3:
        return "purple"
    else:
        return "yellow"

In [ ]:
def visualizza_interattivo_con_pyvis(G, nome_file):

    # Calcolo grado totale e quartili
    if G.is_directed():
        grado = [grado for nodo, grado in G.out_degree()]
    else:
        grado = [grado for nodo, grado in G.degree()]

    grado_tot = sum(grado)
    print(f"Grado totale: {grado_tot}")

    q1, q2, q3 = calcola_quartili(grado)

    # Inizializza PyVis
    net = Network(
        notebook=True, 
        height="1000px", 
        width="100%", 
        directed=G.is_directed()
    )

    # Personalizza i nodi
    for nodo in G.nodes():
        if G.is_directed():
            grado = G.out_degree(nodo)
        else:
            grado = G.degree(nodo)

        size = max(0, 3.5*grado)
        color = colore_nodo(grado, q1, q2, q3)

        net.add_node(
            nodo, 
            size=size, 
            color={
                "background": color,
                "border": "black",
                "highlight" : color
            },
            borderWidth=0.5,
            title=f"Nodo: {nodo}, Grado: {grado}"
        )

    for u, v in G.edges():
        net.add_edge(u, v)
    
    net.set_options("""
var options = {
    "physics": {
        "enabled": true,
        "stabilization": {
            "enabled": true,
            "iterations": 1000
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
            "gravitationalConstant": -50,
            "centralGravity": 0.01,
            "springLength": 150,
            "springConstant": 0.005
        },
        "maxVelocity": 3,
        "minVelocity": 0.1
    },
    "interaction": {
        "navigationButtons": true
    },
    "edges": {
        "color": {
            "color": "lightgrey",
            "highlight": "blue"
        },
        "width": 0.5,
        "selectionWidth": 4
    }
}
""")

    net.show(nome_file)

In [ ]:
visualizza_interattivo_con_pyvis(G, "grafo1_interattivo.html")

In [ ]:
visualizza_interattivo_con_pyvis(G2, "grafo2_interattivo.html")

# ___FASE 6___
# Scelta per la serializzazione delle metriche

Per ciascun grafo (diretto e artificiale), vengono calcolate esclusivamente le misure pertinenti al loro contesto specifico: 
- **Grafo diretto**: vengono calcolate misure dipendenti dalla direzione (PageRank, In-degree, Out-degree). Misure globali come centro e raggio sono escluse se non applicabili (ad esempio, se il grafo non è fortemente connesso).
- **Grafo artificiale (indiretto)**: vengono calcolate tutte le misure globali (centro, raggio, distanza media, ecc.), ma vengono escluse misure come PageRank o In/Out-degree, non pertinenti in un grafo indiretto. 


// Avevo pensato di calcolare tutto adattando alcune misure (tipo centro sulla versione indiretta per il diretto), ma poi mi sembrava più complicato da spiegare e forse meno corretto. Da chiedere al prof:
1. Va bene lasciare fuori le misure che non hanno senso?
2. Posso adattare i calcoli (tipo usare il grafo indiretto) o meglio lasciare null e basta?


In [ ]:
#funzione per stimare la small-worldness con sigma e omega
def calcola_small_worldness(G, niter=5, nrand=10, seed=None):
    sigma_value = sigma(G, niter=niter, nrand=nrand, seed=seed)
    omega_value = omega(G, niter=niter, nrand=nrand, seed=seed)
    return {"sigma": sigma_value, "omega": omega_value}

__GRAFO DIRETTO__

_misure globali_

In [ ]:
distanze = []
for nodo in G.nodes:
    lunghezze = nx.single_source_shortest_path_length(G, nodo)
    distanze.extend(lunghezze.values())

distanza_media = sum(distanze) / len(distanze)
distanza_massima = max(distanze)
clustering_medio = nx.average_clustering(G)
transitivita = nx.transitivity(G)

_misure di centralità_

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
degree_centrality = nx.degree_centrality(G)
in_degree_centrality = nx.in_degree_centrality(G)
out_degree_centrality = nx.out_degree_centrality(G)
page_rank = nx.pagerank(G)

centrality_measures = {}
for nodo in G.nodes:
    centrality_measures[nodo] = {
        "betweenness_centrality": betweenness_centrality.get(nodo, 0),
        "closeness_centrality": closeness_centrality.get(nodo, 0),
        "degree_centrality": degree_centrality.get(nodo, 0),
        "in_degree_centrality": in_degree_centrality.get(nodo, 0),
        "out_degree_centrality": out_degree_centrality.get(nodo, 0),
        "page_rank": page_rank.get(nodo, 0)
    }

In [ ]:
risultati = {
    "global_metrics": {
        "graph_center": ["non calcolabile"],
        "radius": "non calcolabile",
        "average_distance": distanza_media,
        "max_distance": distanza_massima,
        "clustering_coefficient": clustering_medio,
        "transitivity": transitivita,
    },
    "centrality_measures": centrality_measures,
    "small_worldness": "non calcolabile"
}

with open("misure_grafo_diretto.json", "w") as f:
    json.dump(risultati, f, indent=4)

print("Analisi completa salvata in misure_grafo_diretto.json")

_GRAFO ARTIFICIALE_

_misure globali_

In [ ]:
centro = nx.center(G2)
raggio = nx.radius(G2)
distanza_media = nx.average_shortest_path_length(G2)
distanza_massima = nx.diameter(G2)
clustering_medio = nx.average_clustering(G2)
transitivita = nx.transitivity(G2)

_misure di centralità_

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G2)
closeness_centrality = nx.closeness_centrality(G2)
degree_centrality = nx.degree_centrality(G2)

centrality_measures = {}
for nodo in G2.nodes:
    centrality_measures[nodo] = {
        "betweenness_centrality": betweenness_centrality.get(nodo, 0),
        "closeness_centrality": closeness_centrality.get(nodo, 0),
        "degree_centrality": degree_centrality.get(nodo, 0),
        "in_degree_centrality": degree_centrality.get(nodo, 0),
        "out_degree_centrality": degree_centrality.get(nodo, 0),
        "page_rank": "non calcolabile"
    }

In [ ]:
risultati = {
    "global_metrics": {
        "graph_center": centro,
        "radius": raggio,
        "average_distance": distanza_media,
        "max_distance": distanza_massima,
        "clustering_coefficient": clustering_medio,
        "transitivity": transitivita,
    },
    "centrality_measures": centrality_measures,
    "small_worldness": calcola_small_worldness(G2)
}

with open("misure_grafo_artificiale.json", "w") as f:
    json.dump(risultati, f, indent=4)

print("Analisi completa salvata in misure_grafo_artificiale.json")